# Intermediate Neural Network in Keras

Bulding an Intermediate Neural Network to classify MNIST digits.

#### Set seed for reproducibility

In [ ]:
import numpy as np
np.random.seed(42)

#### Load dependencies

In [ ]:
!pip install --upgrade git+https://www.github.com/ekholabs/keras-contrib.git@improve-sinerelu

In [ ]:
import os
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Layer, Activation, Dense, Dropout, LeakyReLU, BatchNormalization
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

from keras import backend as K
from keras.utils.generic_utils import get_custom_objects
from keras_contrib.layers.advanced_activations import SineReLU

#### Load data

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#### Preprocess data
Flatten and normalise input data.

In [ ]:
X_train = X_train.reshape(60000, 784).astype('float32')
X_test = X_test.reshape(10000, 784).astype('float32')

In [ ]:
X_train /= 255
X_test /= 255

In [ ]:
# One-hot encoded categories
n_classes = 10
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

#### Design Neural Network architecture

In [ ]:
e = 0.0055

model = Sequential()
model.add(Dense(128, input_shape = (784,)))
# model.add(BatchNormalization())
# model.add(SineReLU(epsilon=e))
# model.add(Activation('relu'))
# model.add(LeakyReLU(alpha=0.01))
model.add(Dropout(0.2, seed=42))

model.add(Dense(256))
model.add(BatchNormalization())
model.add(SineReLU(epsilon=e))
# model.add(Activation('relu'))
# model.add(LeakyReLU(alpha=0.01))
model.add(Dropout(0.3, seed=42))

model.add(Dense(1024))
model.add(BatchNormalization())
model.add(SineReLU(epsilon=e))
# model.add(Activation('relu'))
# model.add(LeakyReLU(alpha=0.01))
model.add(Dropout(0.5, seed=42))
model.add(Dense(10, activation = 'softmax'))

In [ ]:
model.summary()

#### Callbacks

In [ ]:
modelCheckpoint = ModelCheckpoint(monitor='val_acc', filepath='model_output/weights-deepnet-mnist.hdf5',
                                               save_best_only=True, mode='max')
earlyStopping = EarlyStopping(monitor='val_acc', mode='max', patience=20)


if not os.path.exists('model_output'):
    os.makedirs('model_output')
    
tensorboard = TensorBoard("../new_logs/deepnet-mnist-sinerelu-XI", histogram_freq=1, write_grads=True, write_images=True)


#### Configure model

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

#### Train!

In [ ]:
history = model.fit(X_train, y_train, batch_size = 128, epochs = 50, verbose = 1,
          validation_split = 0.05, callbacks=[modelCheckpoint, earlyStopping, tensorboard])

#### Test Predictions

In [ ]:
saved_model = keras.models.load_model('model_output/weights-deepnet-mnist.hdf5')
predictions = saved_model.predict_classes(X_test, verbose = 2)
np.std(history.history['loss'])

#### Test Final Accuracy

In [ ]:
final_loss, final_acc = saved_model.evaluate(X_test, y_test, verbose = 1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))
final_loss, final_acc = model.evaluate(X_test, y_test, verbose = 1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))